<a href="https://colab.research.google.com/github/Hernan4444/Hernan4444-diplomado-sistemas-recomendadores-2020-1/blob/master/Diplomado_Alumno_2020_Sistemas_Recomendadores_3_Content_Based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica de Sistemas Recomendadores 3: Content based

En este práctico, utilizaremos la biblioteca de Python [sklearn](https://scikit-learn.org/stable/), para aprender sobre 2 algoritmos para recomendación basado en contenidos y de unas herramientas para preprocesar los textos. En particula, este practico verá:

* TF-IDF
* Latent Dirichlet Allocation (LDA)
* Uso de Stop Words
* Non-negative Matrix Factorization (NMF)


**Autor**: Antonio Ossa, Manuel Cartagena y editado por Hernán Valdivieso

**Ayudantes**: Manuel Cartagena y Hernán Valdivieso




Nombre: 

- **Completar**
- **Completar**

**En caso de hacerlo en parejas y no poner ambos nombres repercutirá en un descuento**

# Índice

>[Práctica de Sistemas Recomendadores 3: Content based](#scrollTo=NC-ceGb8LRLT)

>[Índice](#scrollTo=l-3HVp9guEsg)

>[Descargando la información](#scrollTo=IFpEoacrMwQx)

>[Revisar archivos descargados](#scrollTo=TJon9T5ZMwRG)

>>[Preparar entorno](#scrollTo=7HU7NoDUhnYl)

>>[Preprocesamiento de datos](#scrollTo=PUYnjZ1yOY-A)

>>[Actividad 1](#scrollTo=ckrxbKlTUVJ8)

>[Tf-idf](#scrollTo=f23GriULTHgV)

>[LDA](#scrollTo=bxqEz_S0ensc)

>>[Actividad 2](#scrollTo=2i60zO2fgyVF)

>>[Actividad 3](#scrollTo=th7K6SUbhCkf)

>>[Generar recomendaciones](#scrollTo=EuNk3cw3SblR)

>>[Actividad 4](#scrollTo=EXwYjORwTr17)

>>[Actividad 5](#scrollTo=4AUbaWOpoAvc)

>>[Actividad 6](#scrollTo=GfgYZ8SLoFsV)

>[Stop words](#scrollTo=gfInn_xVSmZ6)

>>[Actividad 7](#scrollTo=yaclTipvxM5E)

>>[Actividad 8](#scrollTo=uIfyEAdGxgoK)

>>[Actividad bonus de LDA (opcional)](#scrollTo=ncn1B8W5mPWb)

>[Otro método: Non-negative Matrix Factorization (NMF)](#scrollTo=X60ZefJhxM5c)

>>[Actividad bonus de NMF (Opcional)](#scrollTo=Mz2ybg7KowBA)

>>[Actividad 9 (Obligatoria)](#scrollTo=nl155tv3pMnW)



# Descargando la información

Vaya ejecutando cada celda presionando el botón de **Play** o presionando Ctrl+Enter (Linux y Windows) o Command+Enter (Macosx) para descargar las bases de datos.

*   Recursos:
  * `dictionary.p`
  * `dictionary-stemm.p`
  * `tfidf_model.p`
  * `tfidf_model-stemm.p`
*   Dataset:
  *  `corpus1.csv`

In [0]:
# Descarga de recursos
!curl -L -o 'resources.tar.gz' 'https://github.com/PUC-RecSys-Class/Syllabus/blob/master/Practico%204/files/resources.tar.gz?raw=true'

# Descompresión del archivo
!tar -xvf resources.tar.gz

In [0]:
# Descarga del dataset
!curl -L -o 'dataset.tar.gz' 'https://github.com/PUC-RecSys-Class/Syllabus/blob/master/Practico%204/files/dataset.tar.gz?raw=true'

# Descompresión del archivo
!tar -xvf dataset.tar.gz

# Revisar archivos descargados

Revisemos el _dataset_ descargado:


In [0]:
import pandas as pd

corpus_df = pd.read_csv('./corpus1.csv', sep='\t',
                        header=None, encoding='latin',
                        names=['id', 'title', 'abstract'])
corpus_df.head(5)

Podemos ver que este _dataet: contiene 3 columnas:
* **_id_**: identificador de cada texto
* **_title_**: título del documento, en este caso, de un _paper_
* **_abstract_**: primer párafo del _paper_ que es una representación abreviada, objetiva y precisa del contenido de un documento o recurso, sin interpretación crítica y sin mención expresa del autor del resumen.

## Preparar entorno
Primero es necesario instalar algunas librerías previas

In [0]:
!pip install nltk
!pip install sklearn
!pip install gensim
!pip install pandas
!pip install numpy

Luego necesitamos importar las librerías a utilizar en este práctico. No se asusten por todas las librerías, iremos explicando lo más importante a medida que se avanza en el práctico.

In [0]:
import string

import gensim
import nltk
import numpy as np
import pandas as pd
import sklearn

from collections import Counter
from os.path import isfile
from textwrap import wrap

from gensim import corpora, models, similarities
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

pd.options.display.max_columns = None


## Preprocesamiento de datos

Volvemos a cargar el _dataset_ a utilizar

In [0]:
corpus_df = pd.read_csv('./corpus1.csv', sep='\t',
                        header=None, encoding='latin',
                        names=['id', 'title', 'abstract'])
corpus_df.head(5)

Luego descargamos las librerías de NLTK necesarias:

In [0]:
nltk.download('punkt')

En este momento estamos bajando un _tokenizador_ específico llamado [Punkt Sentence Tokenizer](https://kite.com/python/docs/nltk.tokenize.punkt). Este será usado a continuación para realizar una cierta tarea con los textos (no vamos a decir cual es porque una actividad es que comenten que hace dado unos ejemplos que mostramos c: ). 

Lo siguiente es implementar una función que transforme texto no estructurado a una lista de *tokens* procesados.

In [0]:
def get_tokens(text):
    # Pasar todo a minuscula
    lowers = text.lower()
    
    # Quitar puntuación
    no_punctuation = lowers.translate({ord(c): None for c in string.punctuation})
    
    # Tokenizar 
    tokens = nltk.word_tokenize(no_punctuation)
    
    # Retornar resultado
    return tokens


print(get_tokens("I'm a super student for recommender systems!"))
print(get_tokens("First sentence. Seconde sentence."))

En el código anterior, para ejecutar `nltk.word_tokenize()` era necesario tener descargado _punkt_. 

## Actividad 1

En función a las frases ingresadas y al resultado impreso, ¿Qué significa _Tokenizar_?

**Respuesta:** COMPLETAR

In [0]:
# A cada abstract le aplicamos la función de get_tokens
corpus_df['tokenized_abstract'] = corpus_df.abstract.map(get_tokens)
corpus_df.head(5)

Ahora se tiene que generar un diccionario con todas las palabras del *corpus*. Se recomienda revisar la documentación de gensim y leer cómo usar los diccionarios: [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html)

In [0]:
dict_file = './resources/dictionary.p'

if isfile(dict_file): # Verificar si existe el archivo
    dictionary = corpora.dictionary.Dictionary().load(dict_file)
    
else: # En otro caso, crear el archivo y guardarlo
    dictionary = corpora.dictionary.Dictionary(documents=corpus_df.tokenised_abstract.tolist())
    dictionary.save(dict_file)

In [0]:
# Texto original
print("Texto 1")
wrap(str(corpus_df.loc[0]["tokenized_abstract"]))


In [0]:
# Texto pasado por el diccionario
print("Texto 1")
wrap(str(dictionary.doc2bow(corpus_df.loc[0]["tokenized_abstract"])))


Cuando se hizo `dictionary.doc2bow` se transformó una lista de palabas a un contador de ellas. En donde cada tupla representa `(ID, cantidad de veces)` de modo que se reduce la cantidad de palabras del texto a información numerica. 

Por ejemplo, la tupla `(30, 5)` indica que la palabra con ID 30 está 5 veces en el texto. Revisando el texto podemos ver que la palabra **"a"** es la que está repetida 5 veces. Esto implica que **"a"** está asignada al ID 30.

Ahora aplicaremos esta función a cada texto del _dataset_.

In [0]:
corpus_df['bow'] = corpus_df.tokenized_abstract.map(dictionary.doc2bow)

corpus = corpus_df['bow'].tolist()

corpus_df.head(5)

# Tf-idf

Recordemos que Tf-idf es una medida numérica que expresa cuán relevante es una palabra para un documento en una colección. Ahora, dada la frecuencia de cada palabra en cada texto, se v a utilizar esta ténica para obtener tuplas de la forma `(ID, Tf-idf)` en donde ID será el ID de la palabra igual como estaba antes (por ejemplo **"a"** tiene ID 30) y Tf-Idf será el valor dado por este algoritmo a la palabra en cuestión.

In [0]:
tfidf_model_file = 'resources/tfidf_model.p'

if isfile(tfidf_model_file):
    tfidf_model = models.tfidfmodel.TfidfModel().load(tfidf_model_file)

else:
    tfidf_model = models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary)
    tfidf_model.save(tfidf_model_file)

corpus_df['tf_idf'] = tfidf_model[corpus_df.bow.tolist()]
corpus_df.head(5)

# LDA

A continuación utilizaremos el modelo LDA para identificar 10 tópicos sobre los documentos del dataset:

In [0]:
topic_number = 10
iterations = 200

lda_model = models.LdaModel(corpus, num_topics=topic_number,
                            id2word=dictionary, passes=5, iterations=iterations)

In [0]:
print(lda_model[corpus_df.loc[0].bow])
print(lda_model[corpus_df.loc[1].bow])
print(lda_model[corpus_df.loc[2].bow])
print(lda_model[corpus_df.loc[3].bow])
print(lda_model[corpus_df.loc[4].bow])
print(lda_model[corpus_df.loc[5].bow])

Ahora aplicaremos el algoritmo de LDA para identificar los tópicos a cada documento del _dataset_.

In [0]:
corpus_df['lda'] = lda_model[corpus_df.bow.tolist()]
corpus_df.head(5)

## Actividad 2

**Pregunta:** Explique qué representa la columna `lda`, ¿qué significan cada tupla de números?

**Respuesta:** COMPLETAR

En la siguiente celda se mostrarán 10 tópicos del modelo LDA.

In [0]:
lda_model.print_topics(10)

## Actividad 3

**Pregunta:** ¿Qué representa lo impreso en la celda anterior?

**Respuesta:** COMPLETAR

## Generar recomendaciones

En esta sección se implementan las funciones necesarias para poder generar recomendaciones dado lo que un usuario ha consumido. De manera artificial, se "samplearán" 3 documentos aleatorios que representarán al usuario objetivo (`sample`). Luego tendrás que generar diferentes recomendaciones y evaluar los resultados.

In [0]:
# Random users
samples = corpus_df.sample(3)
samples_ids = []

for n, (ix, paper) in enumerate(samples.iterrows()):
    samples_ids.append(ix)
    idx, title, abstract, bow, tf_idf, lda = paper[[
        'id', 'title', 'abstract', 'bow', 'tf_idf', 'lda']]
    print('%d) %s' % (n+1, title))
    print('')
    print("\n".join(wrap(abstract)))
    print('\n')

Lo anterior son 3 textos tomados al azar. Asumiremso que una persona vió estos 3 textos y ahora vamos a recomendarle **5 nuevos documentos por cada documento visto**.

In [0]:
# Recommendation functions

N = len(dictionary)


def to_sparse(matrix):
    return csr_matrix([
        gensim.matutils.sparse2full(row, length=N)
        for row in matrix
    ])


def make_recommendations(model, metric, neighbors):
    M = len(corpus)

    X = to_sparse(corpus_df[model].tolist())
    document_index = NearestNeighbors(
        n_neighbors=(neighbors + 1),
        algorithm='brute',
        metric=metric).fit(X)
    return document_index


def print_recommendations(indexes, model):
    for n, (ix, paper) in enumerate(samples.iterrows()):
        dists, neighbors = indexes.kneighbors([gensim.matutils.sparse2full(paper[model], length=N)])
        print(paper['title'])
        print('')
        print('Documentos cercanos: ')
        i = 1
        for neighbour in neighbors[0]:
            if ix != neighbour:
                line = str(i) + ". " + corpus_df.iloc[neighbour]['title']
                print(line)
                i += 1
        print('\n')

A continuación deberá utilizar las funciones implementadas anteriormente para generar nuevas recomendaciones variando los parámetros del modelo. **Agregue nuevas celdas para cada implementación y/o pregunta.**


Aquí hay 2 ejemplos, puede crear más celdas para hacer las pruebas necesarias:

In [0]:
# Recommendation example: TF-IDF con distancia euclidean y solo 5 documentos
doc_idx = make_recommendations('tf_idf', 'euclidean', 5)
print_recommendations(doc_idx, 'tf_idf')

In [0]:
# Recommendation example: LDA  con distancia euclidean y solo 3 documentos
doc_idx = make_recommendations('lda', 'euclidean', 3)
print_recommendations(doc_idx, 'lda')

## Actividad 4

**Pregunta:** Ejecute el modelo utilizando como representación tf-idf y una métrica de distancia euclideana. Modifique el tercer argumento `neighbors` a 10. ¿Qué efecto tiene este cambio en el modelo en las recomendaciones observadas aparte de que hay más documentos recomendados? ¿por qué pasa esto? 



In [0]:
# Código para responder la pregunta

**Respuesta:** COMPLETAR

## Actividad 5

**Pregunta:** Eligiendo un valor fijo de neighbors y modelo lda ¿Qué efecto tiene el usar LDA versus TF-IDF en las recomendaciones observadas bajo la misma métrica de distancia?

In [0]:
# Código para responder la pregunta

**Respuesta:** COMPLETAR

## Actividad 6

**Pregunta:** Pruebe nuevamente con LDA usando 5 tópicos y con 20 tópicos ¿qué efecto tiene el número de tópicos en las recomendaciones observadas?

**Importante** Tiene que volver a crear un `LdaModel` pero con 5 tópicos y luego con 20, si hace solo `make_recommendations('lda', 'euclidean', 20)`, **no significa que está usando 20 tópicos** sino que está recomendando 20 documentos. No olvide volver a generar los 3 `samples` que simulan las lecturas del usuario, para que disponga de la columna 'lda' actualizada.

In [0]:
# Código para responder la pregunta

**Respuesta:** COMPLETAR

# Stop words

A continuación, intentaremos mejorar los resultados obtenidos con LDA eliminando las *stopwords*. ¿Qué son las *stopwords*? Son palabras vacías, sin significado, que no aportan (de manera significativa) al sentido de una frase, como los artículos, pronombres, etc.

In [0]:
nltk.download('stopwords')

In [0]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    filtered_words = [
        word for word in text if word not in stopwords.words('english')
    ]
    return filtered_words

Ahora eliminamos los stopwords de los textos y volvemos a hacer todo el proceso pero con textos diferentes. Este proceso dura aproximadamente **5 minutos**

In [0]:
%%time
# Puede que se demore un poco esta celda
corpus_df['tokenized_abstract_without_stopwords'] = corpus_df.tokenized_abstract.map(remove_stopwords)

In [0]:
corpus_df.head(5)

In [0]:
corpus_df['bow_without_stopwords'] = corpus_df.tokenized_abstract_without_stopwords.map(dictionary.doc2bow)
corpus_df.head(5)

In [0]:
corpus = corpus_df['bow_without_stopwords'].tolist()

tfidf_model_file_without_stopwords = 'resources/tfidf_model.p'

if isfile(tfidf_model_file):
    tfidf_model_without_stopwords = models.tfidfmodel.TfidfModel().load(tfidf_model_file)

else:
    tfidf_model_without_stopwords = models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary)
    tfidf_model_without_stopwords.save(tfidf_model_file_without_stopwords)

corpus_df['tf_idf_without_stopwords'] = tfidf_model_without_stopwords[corpus_df.bow_without_stopwords.tolist()]
corpus_df.head(5)

In [0]:
topic_number = 10

lda_model_without_stopwords = models.LdaModel(corpus, num_topics=topic_number, id2word=dictionary, passes=5, iterations=200)
corpus_df['lda_without_stopwords'] = lda_model_without_stopwords[corpus_df.bow_without_stopwords.tolist()]
corpus_df.head(5)

In [0]:
lda_model_without_stopwords.print_topics(10)

## Actividad 7

**Pregunta:** ¿Qué puede comentar de estos nuevos tópicos comparándolos con los obtenidos previamente (sección LDA)?

**Respuesta:**

In [0]:
# Rellocate user

samples = corpus_df.iloc[samples_ids]

for n, (ix, paper) in enumerate(samples.iterrows()):
    idx, title, abstract, bow, tf_idf, lda, bow_without_stopwords, tf_idf_without_stopwords, lda_without_stopwords = paper[[
        'id', 'title', 'abstract', 'bow', 'tf_idf', 'lda', 'bow_without_stopwords', 'tf_idf_without_stopwords', 'lda_without_stopwords']]
    print('%d) %s' % (n+1, title))
    print('')
    print("\n".join(wrap(abstract)))
    print('\n')

In [0]:
# Recommendation example: TF-IDF without stopwords

doc_idx = make_recommendations('tf_idf_without_stopwords', 'euclidean', 5)
print_recommendations(doc_idx, 'tf_idf_without_stopwords')

In [0]:
# Recommendation example: LDA without stopwords

doc_idx = make_recommendations('lda_without_stopwords', 'euclidean', 5)
print_recommendations(doc_idx, 'lda_without_stopwords')

## Actividad 8

**Pregunta:** ¿Cómo cambian las recomendaciones entre ambos métodos ahora que no consideramos las *stopwords*?

**Respuesta:**

## Actividad bonus de LDA (opcional)

**Pregunta:** Realice el siguiente gráfico. Pruebe graficando los items con respecto al tópico con mayor probabilidad de pertenencia, para poder hacer el gráfico deberá usar algún método de reducción de dimensionalidad como PCA o T-SNE a los valores de LDA que están en el dataframe.

**Importante** Priorice realizar las demás actividades obligatorias antes de esta. En esta lo que tiene que hacer es 
1. Tomar los valores de LDA  (tomar la columna adecuada)
2. Aplicar reducción de dimensión a esos datos para 2 componentes
3. Graficar cada punto y luego pintar según el tópico con mayor probabilidad.

Ejemplo:

![Expected plot](https://raw.githubusercontent.com/PUC-RecSys-Class/Syllabus/master/Practico%204/files/plot.png)

**Importante:** Sigan bajando en el práctico porque hay otras actividades abajo. 

In [0]:
# Codigo para generar el grafico

# Otro método: Non-negative Matrix Factorization (NMF)

A continuación, utilizaremos el modelo NMF para generar recomendaciones:

In [0]:
n_components = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Tópico #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
        print()
    print()


data_samples = corpus_df['abstract'].values

# Formato TF-IDF de sklearn
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data_samples)

# Fit NMF
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5,
          ).fit(tfidf)
nmf_transform_1 = nmf.transform(tfidf)
# Display NMF
print("Tópicos:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)
print()
print()

# Fit NMF with KL-Divergence
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5,
          ).fit(tfidf)
# Display NMF with KL-Divergence
print("Tópicos con divergencia KL:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)
print()
print()

Agregamos los valores obtenidos como una columna en el *dataframe*:

In [0]:
corpus_df['NMF'] = [[(i, prob) for i, prob in enumerate(l)] for l in nmf_transform_1.tolist()]

## Actividad bonus de NMF (Opcional)
**Pregunta:** En la siguiente casilla de código (`corpus_df.loc[0]['NMF']`) se imprime una lista de tuplas para el documento 0, ¿qué significan las tuplas de dicha lista?

**Respuesta:**

**Importante**: Siga bajando porque hay una última actividad obligatoria.

In [0]:
corpus_df.loc[0]['NMF']

Ahora observaremos todas las columnas que tiene el _dataframe_. Pueden ver que en la última está `NMF` que es la nueva columna agregada.

In [0]:
corpus_df.head()

Repetimos el proceso de *sampling* para inspeccionar los resultados:

In [0]:
# Rellocate user

samples = corpus_df.iloc[samples_ids]

for n, (ix, paper) in enumerate(samples.iterrows()):
    idx, title, abstract, bow, tf_idf, lda, bow_without_stopwords, tf_idf_without_stopwords, lda_without_stopwords, nmf = paper[[
        'id', 'title', 'abstract', 'bow', 'tf_idf', 'lda', 'bow_without_stopwords', 'tf_idf_without_stopwords', 'lda_without_stopwords', 'NMF']]
    print('%d) %s' % (n+1, title))
    print('')
    print("\n".join(wrap(abstract)))
    print('\n')

In [0]:
doc_idx = make_recommendations('NMF', 'euclidean', 5)
print_recommendations(doc_idx, 'NMF')

## Actividad 9 (Obligatoria)

**Pregunta:** Compare y comente sobre las recomendaciones hechas por los métodos anteriores con las obtenidas usando NMF.

**Respuesta:**